In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
df = pd.read_excel('Report_JR14_Vehicle History Report - data.xlsx',skiprows=10)

In [3]:
'''
    Create Trips from the GPS Data
    1. 
'''
df.columns

Index(['GPS', 'Device', 'Server', 'Reason', 'Driver', 'Desc.', 'Rego',
       'Latitude', 'Longitude', 'Direction', 'Unnamed: 10', 'Speed', 'RawGPS',
       'Status', 'Flags', 'User Defined', 'Input', 'Output', 'Light', 'Button',
       'Max', 'Average', 'Spd Acc', 'Samples', 'Map Ref', 'Suburb', 'Distance',
       'Position', 'Eng Cool Temp', 'Eng Oil Temp', 'OilPressure',
       'ECM Economy', 'Trip', 'Total Used', 'RPM', 'Tot Hour', 'Brake Hits',
       'Odometer', 'MaxF', 'MaxB', 'MaxLR', 'MaxZ', 'Position.1', '  1  ',
       '  2', '  3  ', '  4  ', '  5  ', '  6  ', '  7  ', '  8  ', '  9  ',
       '  10  ', 'Volts', 'Seconds', 'Speed.1', 'Zone 1', 'Zone 2', 'Zone 3',
       'Zone 4', 'Zone 1.1', 'Zone 2.1', 'Zone 3.1', 'Zone 4.1', '  1  .1',
       '  2  ', '  3  .1', '  4  .1', '  5  .1', 'Vehicle', 'Auxiliary',
       '  1  .2', '  2  .1', '  3  .2', '  4  .2', '  5  .2', 'Mass', 'Limit',
       'HDOP', 'Satellites', 'Temperature', 'From Flash', 'Offset Time',
       'Offset'

In [7]:
'''
    Data Processing:

    Alpha: Check if Odometer Offset exists - Current logic searches for text Odometer Offset in Suburb. Best is to look in 'Offset' Column and get unique values
        Y: process add previous rows odo reading, add offset -- STATUS 1 ***OR***
        Y: filter the rows from the next day -- STATUS 2
        N: proceed usually. 
    1. Filter required columns out of overall dataset
    2. Concat Lat+Long into coords for each row
'''
# def manipulate_Odometer_Offsets(df,status):

#     odometer_offsets = df[df['Reason'].str.contains("Odometer Offset")].index.tolist()
    
#     # Basic Checks
#     # print(odometer_offsets)
#     if len(odometer_offsets) != 0 or status == 3:
#         return df
#     if status == 1:
#         # Write logic for taking previous rows and add the offset value
#         start = 0
#         for offset in range(len(odometer_offsets)):
#             end = odometer_offsets[offset]
#             value = df.loc[end]['Odometer']-df.loc[end-1]['Odometer']
#             df.loc[start:end]['Odometer'] += value
#             start = end
#         return df
#     elif status == 2:
#         date = None
#         for offset in range(len(odometer_offsets)):
#             date = pd.to_datetime(df.loc[odometer_offsets[offset]]['GPS']).date()
#             date = date + timedelta(days = 1)
#         return df[df['GPS'].dt.date >= date]
    
# Donot run odometet manipulator right now!
# df1 =manipulate_Odometer_Offsets(df,2)
def all_preprocessors(df):
    df['Date'] = pd.to_datetime(df['GPS'],format="%Y-%m-%d").dt.date
    df = df[['GPS', 'Device', 'Server', 'Reason', 'Driver', 'Desc.','Speed', 'Rego','Latitude', 'Longitude', 'Direction','Odometer', 'MaxF', 'MaxB', 'MaxLR', 'MaxZ','Offset Time','Offset']]
    df.loc[:, 'coords'] = df['Latitude'].astype(str) + ',' + df['Longitude'].astype(str)
    return df
    # return manipulate_Odometer_Offsets(df,2)

df = all_preprocessors(df)
# df = manipulate_Odometer_Offsets(df,1)

D:\AppData\Local\Temp\ipykernel_20468\907986824.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['Latitude'].astype(str) + ',' + df['Longitude'].astype(str)


In [5]:
'''
    Read and process a list of dates for calendar file.
 
'''

def create_calendar_file(df):
    dates = df['GPS'].dt.date.unique()

    calendardates = [
        {'date': item, 'day_of_week': item.strftime('%A')} for item in dates
    ]
    calendar_cols = ['service_id','start_date','end_date','monday','tuesday','wednesday','thursday','friday','saturday','sunday']
    calendar = pd.DataFrame(columns=calendar_cols)

    for date in calendardates:
        day = date['day_of_week']
        idate = date['date'].strftime('%Y%m%d')
        if  day == 'Monday':
            data = pd.DataFrame([[idate,idate,idate,'1','0','0','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data],ignore_index=True)
        elif day == 'Tuesday':
            data = pd.DataFrame([[idate,idate,idate,'0','1','0','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Wednesday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','1','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Thursday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','1','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Friday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','1','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Saturday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','0','1','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Sunday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','0','0','1']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)

    if len(calendar) != len(dates):
        print('Some dates have not been exported to Calendar')
    else:
        return calendar

# TEST THE FUNCTION HERE
# print(create_calendar_file(df))


In [15]:
'''

    Create Routes from Trips
    
    1. Get all Ignition On and Off events
    2. If distance between consecutive Ignition On and Off event < 1 km

    
    A. Location of Ignition On and Off, subtraction (Off['odomter'] - On['Odmeter']) > 1 -> Push to list1. Push all rows between On and Off to a new list.
    B. df[df['Reason'].str.contains("Odometer Offset")].index.tolist() To find if odometer offset exists, .loc['Offset'] integer
    C. Use the integrer val add [0:loc[odometer_offset]]
'''


session_starts = []
session_ends = []
current_session_start = None
on_odometer = None
off_odometer = None

for index, row in df.iterrows():
    if row['Reason'] == 'Ignition On':
        current_session_start = index
        on_odometer = row['Odometer']
    elif row['Reason'] == 'Ignition Off' and current_session_start is not None:
        off_odometer = row['Odometer']
        if on_odometer is not None and off_odometer is not None:
            diff = off_odometer - on_odometer
            if diff > 1:
                session_starts.append(current_session_start)
                session_ends.append(index)
                current_session_start = None
        else:
            # Do something when the Ignition On or Off row does not have any odometer value
            print("Ignition On or Off do nt have an Odometer Value")

# If a session ends without 'Ignition Off,' you may want to check and handle it outside the loop.

session_dataframes = []

# Iterate through the session start and end indices to extract sessions
###### I might get the size of session_dataframes di
for start, end in zip(session_starts, session_ends):
    session_df = df.loc[start:end]
    session_dataframes.append(session_df)

for session_df in session_dataframes:
    offset_index = session_df[session_df['Reason'] == 'Odometer Offset'].index
    if len(offset_index)!=0:
        start = 0
        for item in range(len(offset_index)):
            session_df.loc[start:item,'Odometer'] += session_df.iloc[item]['Offset']
            # print(offset_index[item])
            # print(session_df)





    # print(session_df.iloc[0]['Odometer']) # - session_df.iloc[end]['Odometer'] > 0)
    # session_dataframes.append(session_df)

# filtered_df = df.loc[df['Reason'].isin(['Ignition On','Ignition Off'])]

                     GPS              Device                  Server  \
1449 2023-04-04 08:18:57 2023-04-04 08:18:57 2023-04-03 22:19:01.627   
1450 2023-04-04 08:19:24 2023-04-04 08:19:24 2023-04-03 22:19:28.613   
1451 2023-04-04 08:19:59 2023-04-04 08:19:59 2023-04-03 22:20:03.623   
1452 2023-04-04 08:20:59 2023-04-04 08:20:59 2023-04-03 22:21:18.510   
1453 2023-04-04 08:20:59 2023-04-04 08:20:59 2023-04-03 22:21:18.530   
...                  ...                 ...                     ...   
1557 2023-04-04 08:48:23 2023-04-04 08:48:23 2023-04-03 22:48:27.607   
1558 2023-04-04 08:48:27 2023-04-04 08:48:27 2023-04-03 22:48:31.577   
1559 2023-04-04 08:48:32 2023-04-04 08:48:32 2023-04-03 22:48:36.637   
1560 2023-04-04 08:48:37 2023-04-04 08:48:37 2023-04-03 22:48:41.617   
1561 2023-04-04 08:48:48 2023-04-04 08:48:48 2023-04-03 22:48:53.777   

              Reason         Driver  \
1449     Ignition On  Danny Chapman   
1450   Status Report  Danny Chapman   
1451   Status Repo

In [ ]:
# RUN EVERYTHING FROM HERE
